## **2.3 学习正则表达式 re**

### 2.3.1 为什么使用正则表达式？
典型的搜索和替换操作要求您提供与预期的搜索结果匹配的确切文本。虽然这种技术对于对静态文本执行简单搜索和替换任务可能已经足够了，但它缺乏灵活性，若采用这种方法搜索动态文本，即使不是不可能，至少也会变得很困难。

通过使用正则表达式，可以：

    - 测试字符串内的模式。
        例如，可以测试输入字符串，以查看字符串内是否出现电话号码模式或信用卡号码模式。这称为数据验证。
    - 替换文本。
        可以使用正则表达式来识别文档中的特定文本，完全删除该文本或者用其他文本替换它。
    - 基于模式匹配从字符串中提取子字符串。
        可以查找文档内或输入域内特定的文本。
        可以使用正则表达式来搜索和替换标记。

#### 使用正则表达式的优势是什么？  **简洁**
- 正则表达式是用来简洁表达一组字符串的表达式
- 正则表达式是一种通用的字符串表达框架
- 正则表达式是一种针对字符串表达“简洁”和“特征”思想的工具
- 正则表达式可以用来判断某字符串的特征归属

#### 正则表达式在文本处理中十分常用：
- 表达文本类型的特征（病毒、入侵）
- 同时查找或替换一组字符串
- 匹配字符串的全部或部分(主要)

### 2.3.2 正则表达式语法

正则表达式语法由字符和操作符构成:

- 常用操作符
    - `.` 表示任何单个字符
    - `[ ]` 字符集，对单个字符给出取值范围 ，如`[abc]`表示a、b、c，`[a‐z]`表示a到z单个字符
    - `[^ ]` 非字符集，对单个字符给出排除范围 ，如`[^abc]`表示非a或b或c的单个字符
    - `*` 前一个字符0次或无限次扩展，如abc* 表示 ab、abc、abcc、abccc等 
    - `+` 前一个字符1次或无限次扩展 ，如abc+ 表示 abc、abcc、abccc等 
    - `?` 前一个字符0次或1次扩展 ，如abc? 表示 ab、abc
    - `|` 左右表达式任意一个 ，如abc|def 表示 abc、def

    - `{m}` 扩展前一个字符m次 ，如ab{2}c表示abbc
    - `{m,n}` 扩展前一个字符m至n次（含n） ，如ab{1,2}c表示abc、abbc
    - `^` 匹配字符串开头 ，如^abc表示abc且在一个字符串的开头
    - `$` 匹配字符串结尾 ，如abc$表示abc且在一个字符串的结尾
    - `( )` 分组标记，内部只能使用 | 操作符 ，如(abc)表示abc，(abc|def)表示abc、def
    - `\d` 数字，等价于`[0‐9]`
    - `\w` 单词字符，等价于`[A‐Za‐z0‐9_]`

### 2.3.3 正则表达式re库的使用

- 调用方式：import re
- re库采用raw string类型表示正则表达式，表示为：r'text'，raw string是不包含对转义符再次转义的字符串;

#### re库的主要功能函数：

- re.search() 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象 
    - re.search(pattern, string, flags=0)
- re.match() 从一个字符串的开始位置起匹配正则表达式，返回match对象
    - re.match(pattern, string, flags=0)
- re.findall() 搜索字符串，以列表类型返回全部能匹配的子串
    - re.findall(pattern, string, flags=0)
- re.split() 将一个字符串按照正则表达式匹配结果进行分割，返回列表类型
    - re.split(pattern, string, maxsplit=0, flags=0)
- re.finditer() 搜索字符串，返回一个匹配结果的迭代类型，每个迭代元素是match对象
    - re.finditer(pattern, string, flags=0)
- re.sub() 在一个字符串中替换所有匹配正则表达式的子串，返回替换后的字符串
    - re.sub(pattern, repl, string, count=0, flags=0)

    -  flags : 正则表达式使用时的控制标记：
        - re.I -->  re.IGNORECASE :  忽略正则表达式的大小写，`[A‐Z]`能够匹配小写字符
        - re.M -->  re.MULTILINE :  正则表达式中的^操作符能够将给定字符串的每行当作匹配开始
        - re.S -->  re.DOTALL   :  正则表达式中的.操作符能够匹配所有字符，默认匹配除换行外的所有字符
        
#### re库的另一种等价用法

- pat = re.compile(r'[1-9]\d{5}') # 多次使用正则表达式
- rst = pat.search('BIT 1000081')

#### 将正则表达式的字符串形式编译成正则表达式对象(编译）
regex = re.compile(pattern, flags=0)
#### re 库的贪婪匹配和最小匹配

- `.*` Re库默认采用贪婪匹配，即输出匹配最长的子串
- `*?` 只要长度输出可能不同的，都可以通过在操作符后增加?变成最小匹配

### 2.3.4 实战：淘宝商品比价定向爬虫

- 爬取网址：https://s.taobao.com/search?q=书包&js=1&stats_click=search_radio_all%25
- 爬取思路：
    1. 提交商品搜索请求，循环获取页面
    2. 对于每个页面，提取商品名称和价格信息
    3. 将信息输出到屏幕上

In [1]:
# 导入包
import requests
import re

#### 1. 提交商品搜索请求，循环获取页面

In [13]:
def getHTMLText(url):
    """
    请求获取html，（字符串）
    :param url: 爬取网址
    :return: 字符串
    """
    try:
        # 添加头信息,
        kv = {
            'cookie': '_samesite_flag_=true; cookie2=1996e9821457be2f132954719e8587fb; t=97638bcd2f514572833173e70bf0e258; _tb_token_=3387f5fa63357; cna=cGQoF7A9JgcCAbfHr1dQ5CmX; unb=2274370162; lgc=%5Cu5475%5Cu5475%5Cu5F00%5Cu5FC3%5Cu5B9D%5Cu8D1D5656; cookie17=UUpmmLuvV5yrqQ%3D%3D; dnk=%5Cu5475%5Cu5475%5Cu5F00%5Cu5FC3%5Cu5B9D%5Cu8D1D5656; tracknick=%5Cu5475%5Cu5475%5Cu5F00%5Cu5FC3%5Cu5B9D%5Cu8D1D5656; _l_g_=Ug%3D%3D; sg=623; _nk_=%5Cu5475%5Cu5475%5Cu5F00%5Cu5FC3%5Cu5B9D%5Cu8D1D5656; cookie1=AQI6BmM%2FOn44c9i4S8anRHhzeZhxxznzsxK6lUMYhOo%3D; sgcookie=EPqxFPTaXtgBjx3AQMZLr; uc3=vt3=F8dBxGR1SmOjLphmJ1M%3D&nk2=2WdxW%2FAW0ZBCiCET3ROgbQ%3D%3D&id2=UUpmmLuvV5yrqQ%3D%3D&lg2=Vq8l%2BKCLz3%2F65A%3D%3D; csg=12697a17; skt=17b51de12174c76c; existShop=MTU4NzYzOTkzNw%3D%3D; uc4=nk4=0%4024HDYGcUZ6zjgL1iYLVfIPeNJNsBiaaMQ1Tb&id4=0%40U2gsFBUkCZz0W1Lzl9Tc7m95DDFP; _cc_=UIHiLt3xSw%3D%3D; enc=MMwHiI2EWd4DRCEvsHJfAExNSZ2XChxnA9MF%2Bh%2B52U2mC92BKEke%2B50DJh9Jv4K4QHzSRKOTqyYCJtPO2pXa5w%3D%3D; tfstk=cphGBicg6Ay623HsPNNsk3ByqN-da3_af8eETf8BHOy4wplU7sjCLTzJYGZEjjBf.; hng=CN%7Czh-CN%7CCNY%7C156; thw=cn; uc1=cookie16=VFC%2FuZ9az08KUQ56dCrZDlbNdA%3D%3D&cookie21=WqG3DMC9Fb5mPLIQo9kR&cookie15=VT5L2FSpMGV7TQ%3D%3D&existShop=false&pas=0&cookie14=UoTUPcqfFeXwEg%3D%3D; mt=ci=46_1; v=0; JSESSIONID=1F689FF4DC2240FC1325093448CE4E5C; isg=BAwM3z9dvv4VCKqCYMyRNqLi3Wo-RbDv5GvKg2bNsbda8a37jFeof28DkflJuehH; l=eBEVC63uQ2cgRVSkBO5anurza77TNIRb8sPzaNbMiIHca69d1FNFVNQccLypRdtjgtCA4exrjGibiRek514dg2HvCbKrCyCoexvO',
            'user-agent': 'Mozilla/5.0'
        }
        r = requests.get(url, timeout=30, headers=kv)
        # r = requests.get(url, timeout=30)
        # print(r.status_code)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return "爬取失败"


#### 2. 对于每个页面，提取商品名称和价格信息

In [14]:
def parsePage(glist, html):
    '''
    解析网页，搜索需要的信息
    :param glist: 列表作为存储容器
    :param html: 由getHTMLText()得到的
    :return: 商品信息的列表
    '''
    try:
        # 使用正则表达式提取信息
        price_list = re.findall(r'\"view_price\"\:\"[\d\.]*\"', html)
        name_list = re.findall(r'\"raw_title\"\:\".*?\"', html)
        for i in range(len(price_list)):
            price = eval(price_list[i].split(":")[1])  #eval（）在此可以去掉""
            name = eval(name_list[i].split(":")[1])
            glist.append([price, name])
    except:
        print("解析失败")

#### 3. 将信息输出到屏幕上

In [15]:
def printGoodList(glist):
    tplt = "{0:^4}\t{1:^6}\t{2:^10}"
    print(tplt.format("序号", "商品价格", "商品名称"))
    count = 0
    for g in glist:
        count = count + 1
        print(tplt.format(count, g[0], g[1]))

In [16]:
# 根据页面url的变化寻找规律，构建爬取url
goods_name = "书包"  # 搜索商品类型
start_url = "https://s.taobao.com/search?q=" + goods_name
info_list = []
page = 3  # 爬取页面数量

In [17]:
count = 0
for i in range(page):
    count += 1
    try:
        url = start_url + "&s=" + str(44 * i)
        html = getHTMLText(url)  # 爬取url
        parsePage(info_list, html) #解析HTML和爬取内容
        print("\r爬取页面当前进度: {:.2f}%".format(count * 100 / page), end="")  # 显示进度条
    except:
        continue

爬取页面当前进度: 100.00%

In [18]:
printGoodList(info_list)

 序号 	 商品价格 	   商品名称   
 1  	138.00	电脑包大书包大学生女生背包大双肩包大容量
 2  	59.00 	双肩包男士大容量旅行电脑背包时尚潮流高中初中学生书包女大学生
 3  	149.00	鳄鱼男士双肩包商务休闲电脑帆布背包旅游旅行包时尚潮流学生书包
 4  	279.00	Jordan 官方 AIR JORDAN 双肩包 书包背包
 CW7699
 5  	129.00	Skechers斯凯奇2020春夏青少年男女童实用休闲双肩包书包L120K048
 6  	119.00	Skechers斯凯奇2020春夏青少年男女童实用书包休闲双肩包L419K050
 7  	669.00	kipling女士帆布背包2020年新款时尚简约休闲潮流书包双肩包|ROSE
 8  	99.90 	迪卡侬户外双肩背包男女休闲双肩包运动学生小书包轻便10L QUBP
 9  	689.00	背包双肩包男士商务旅行背包防盗电脑包休闲书包男多功能大旅游包
 10 	175.00	不莱玫迪士尼合作款书包女韩版高中百搭ins双肩包时尚可爱小背包
 11 	149.00	花花公子男士背包2020年新款商务电脑双肩包高中学生大容量书包
 12 	408.00	李宁CF溯系列双肩包敦煌拓男包女包2020新款学生书包运动包
 13 	255.00	Viney双肩包包2020新款潮真皮时尚背包女大容量书包韩版百搭女包
 14 	219.00	NIKE耐克双肩包男包女包官网旗舰高中初中运动包学生书包旅行背包
 15 	998.00	Herschel Little America经典色旅游双肩包男女士背包书包10020
 16 	899.00	【买1送1】Fion/菲安妮大牌双肩包女 2020新款印花背包旅行书包
 17 	358.00	JanSport旗舰店官网女双肩背包学生书包电脑包男背包 3P6X008
 18 	479.00	Fjallraven/瑞典北极狐双肩包kanken书包女电脑旅行背包官方23510
 19 	1098.00	Herschel Dawson大号时尚双肩包 Surplus系列休闲背包书包10649
 20 	409.00	初弎牛仔善恶 学生书包街头潮流男女时尚休闲双肩包男士背包41413
 21 	398.00	安踏中国对背包男潮牌2020春季新